# Core Functionality (Required)
## 1. Resume Ingestion and Processing
- Index multiple resumes: Ingest and process multiple resume files (PDF and plain text).
- Parse content: Extract text from resumes.
- Chunk text: Split parsed text into meaningful chunks (100–500 tokens with overlap).
- Generate embeddings: Produce vector embeddings for each chunk using a free-tier
model.

In [1]:
print("Hello :D")

Hello :D


## 2. Vector Database Integration
- Store embeddings: Save generated embeddings in a vector database.
- Efficient retrieval: Implement top‑K similarity search for queries.

In [2]:
print("Hello :D")

Hello :D


## 3. Retrieval-Augmented Generation (RAG) Chatbot
- Job description input: Accept a text description of the job role.
- Retrieve relevant chunks: Perform vector search to find top resume chunks.
- LLM-based matching: Use an LLM to generate conversational answers about
candidate fit, citing retrieved content.
- Conversational interface: Support follow-up questions for deeper insights.

In [3]:
print("Hello :D")

Hello :D


# Optional Bonus Features
## A. Web User Interface
- Frontend (Bonus): Implement a simple web UI for file upload, job description entry, and
chat interaction.
- Tech Stack: Next.js with TypeScript
- Hosting: Vercel (Hobby/Free Tier)

In [4]:
print("Hello :D")

Hello :D


## B. SQL-Based Metadata Search
- Extract metadata: Tag resumes with structured metadata (skills, titles, experience).
- Metadata storage: Save tags in a relational database (e.g., PostgreSQL alongside
vector store).
- Metadata API: Expose an endpoint for SQL queries (e.g., SELECT * FROM
resume_metadata WHERE skills @> ARRAY['TypeScript'] AND
years_experience >= 5).

In [5]:
print("Hello :D")

Hello :D
